In [1]:
%load_ext autoreload

%autoreload 2

In [3]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2
from faker import Faker
import constants as cons

# Reading configuration file and credentials

In [8]:
config = configparser.ConfigParser()

config.read(cons.CONFIG_FILE)

['config_mysql_aws.cfg']

In [10]:
aws_rds_conn = boto3.client('rds', aws_access_key_id=config.get(cons.USER, cons.ACCESS_KEY),
                            aws_secret_access_key=config.get(cons.USER, cons.SECRET_KEY),
                            region_name='us-east-1'
                            )

In [11]:
rds_instances_ids = []
aws_response = aws_rds_conn.describe_db_instances()

for db_instance in aws_response['DBInstances']:
    db_instance_identifier = db_instance['DBInstanceIdentifier']
    db_engine = db_instance['Engine']
    if (db_engine == 'mysql'):
      aws_mysql = db_instance
    else:
      aws_postgres = db_instance
    print(f"La instancia '{db_instance_identifier}' utiliza el motor de base de datos '{db_engine}'")


La instancia 'ja3plqub98cr' utiliza el motor de base de datos 'mysql'
La instancia 'xux3br0jo1lp' utiliza el motor de base de datos 'postgres'


In [30]:
import mysql.connector

In [31]:
import ddl

In [32]:
try:
    conexionMysql = mysql.connector.connect(
		host=aws_mysql['Endpoint']['Address'],  # Cambia esto por la dirección IP o el nombre del host de tu servidor MySQL
		user=config.get(cons.DB, cons.DB_USERNAME),  # Cambia esto por tu nombre de usuario de MySQL
		password=config.get(cons.DB, cons.DB_PASSWORD),  # Cambia esto por tu contraseña de MySQL
		database=config.get(cons.DB, cons.DB_NAME) # Cambia esto por el nombre de tu base de datos
	)
    cursor = conexionMysql.cursor()
    cursor.execute(ddl.DDL_QUERY, multi=True)
except Exception as ex:
    print("Error!!: ", ex)

